In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import os
import multiprocessing
import pandas as pd
import textract
import boto3
import botocore
from multiprocessing.dummy import Pool as ThreadPool
from src import etl as etl

## Importamos la información

Importamos los datos que se encuentran en formato parquet.

In [3]:
path = 's3://inai-summerofdata/parquet/legacy/inai_step1.parquet'
df = pd.read_parquet(path, engine='pyarrow')
pd.options.display.max_columns = None

Revisaremos las caracteristicas de nuestro dataframe.

In [4]:
df.head()

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,FECHALIMITE,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta
0,1857500001912,2012/01/01 05:04:07,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Copia de los documentos y los datos con que cu...,None,None,Entrega por Internet en el INFOMEX,2012/03/02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0
1,1857500002012,2012/01/01 05:13:16,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Del ATG para precisar información y datos usa...,None,None,Entrega por Internet en el INFOMEX,2012/03/02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0
2,0001800000712,2012/01/01 09:46:39,SECRETARÍA DE ENERGÍA,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,Inexistencia de la información solicitada,EL ASUNTO SE TURNÓ A LA DIRECCIÓN GENERAL DE R...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Energía,zip,2.0
3,0000600003712,2012/01/01 10:11:41,SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,La información está disponible públicamente,Direcci&oacute;n de Internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Hacienda y Crédito Público,pdf,1.0
4,0001500002812,2012/01/01 10:32:15,SECRETARÍA DE DESARROLLO AGRARIO TERRITORIAL ...,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,Entrega de información en medio electrónico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/01,México,Distrito Federal,MIGUEL HIDALGO,11230,Reforma Agraria,zip,4.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1559012 entries, 0 to 1559011
Data columns (total 22 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   FOLIO                        1559012 non-null  object 
 1   FECHASOLICITUD               1559012 non-null  object 
 2   DEPENDENCIA                  1559012 non-null  object 
 3   ESTATUS                      1559012 non-null  object 
 4   MEDIOENTRADA                 1559012 non-null  object 
 5   TIPOSOLICITUD                1559012 non-null  object 
 6   DESCRIPCIONSOLICITUD         1559003 non-null  object 
 7   OTROSDATOS                   878424 non-null   object 
 8   ARCHIVOADJUNTOSOLICITUD      180445 non-null   object 
 9   MEDIOENTREGA                 1559011 non-null  object 
 10  FECHALIMITE                  1392031 non-null  object 
 11  RESPUESTA                    1559012 non-null  object 
 12  TEXTORESPUESTA               1457777 non-n

## Extracción de texto haciendo paralelización de tareas.

Identificamos con cuantos cores cuenta nuestra máquina para aprovechar el máximo.

In [6]:
cores = multiprocessing.cpu_count()
pool = ThreadPool(cores)
cores

12

Creamos tuplas de los parámetros de la función que paralelizaremos.

In [7]:
tuples = [(folio, ext) for folio, ext in zip(df['FOLIO'], df['tipo_archivo_respuesta'])]

Ejecutamos la función para crear la columna texto_respuesta_adjunto que es la que contendra el texto extraído de los archivos pdf.

In [8]:
df['texto_respuesta_adjunto'] = pool.starmap(etl.TextractPDF, tuples)
pool.close()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df.head()

,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,MEDIOENTREGA,FECHALIMITE,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012/01/01 05:04:07,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Copia de los documentos y los datos con que cu...,None,None,Entrega por Internet en el INFOMEX,2012/03/02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,PEMEX EXPLORACION Y PRODUCCION O Oficio...
1,1857500002012,2012/01/01 05:13:16,PEMEX EXPLORACIÓN Y PRODUCCIÓN,Terminada,Electrónica,Información Pública,Del ATG para precisar información y datos usa...,None,None,Entrega por Internet en el INFOMEX,2012/03/02,Entrega de información en medio electrónico,Se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/03/02,México,Distrito Federal,BENITO JUAREZ,3100,Energía,pdf,1.0,""" 2 PEMEX EXPLORACION Y PRODUCCION O ..."
2,0001800000712,2012/01/01 09:46:39,SECRETARÍA DE ENERGÍA,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,Inexistencia de la información solicitada,EL ASUNTO SE TURNÓ A LA DIRECCIÓN GENERAL DE R...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Energía,zip,2.0,None
3,0000600003712,2012/01/01 10:11:41,SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,La información está disponible públicamente,Direcci&oacute;n de Internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/02,México,Distrito Federal,MIGUEL HIDALGO,11230,Hacienda y Crédito Público,pdf,1.0,\xc2\xbfADOS SECRETARIA DE HACIENDA $ do Y CRE...
4,0001500002812,2012/01/01 10:32:15,SECRETARÍA DE DESARROLLO AGRARIO TERRITORIAL ...,Terminada,Electrónica,Información Pública,Agradeceré se me proporcione en archivo magnét...,None,None,Entrega por Internet en el INFOMEX,2012/02/02,Entrega de información en medio electrónico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012/02/01,México,Distrito Federal,MIGUEL HIDALGO,11230,Reforma Agraria,zip,4.0,None


## Guardamos en formato parquet en S3

In [10]:
s3_url = f's3://inai-summerofdata/parquet/inai.parquet'
df.astype({'FOLIO': str, 'CODIGOPOSTAL': str}).to_parquet(s3_url, engine='auto', compression='snappy')